# Herbarium vs Field Classification with **facebook/vit-mae-large** (MAE)

This notebook fine-tunes the MAE encoder from `facebook/vit-mae-large` as a classifier using your dataset lists:

- **Train**: `train.txt` (each line: `relative/image/path label_id`)
- **Valid**: constructed from `test.txt` (each line: `relative/image/path`, no label) paired with `groundtruth.txt` (`relative/image/path label_id`)

Domain-specific augmentations:
- **Herbarium (heavy)**: `RandomResizedCrop(224, scale=(0.8,1.0))`, `ColorJitter(0.4,0.4,0.4,0.2)`, `RandomHorizontalFlip`, `RandomVerticalFlip`, `RandomRotation(45)`
- **Photo (light)**: `RandomResizedCrop(224)`, `RandomHorizontalFlip`

> Set `DATASET_ROOT` to the folder **containing** the paths referenced by the txt files (typically a folder that contains `train/`, `test/`, etc.).

# Environment & Imports

In [4]:
# If needed, install dependencies (uncomment):
# %pip install -U torch torchvision transformers tqdm scikit-learn matplotlib

In [5]:
import os, math, json, random, time
from pathlib import Path
from typing import List, Tuple, Dict
import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

from transformers import AutoImageProcessor, ViTMAEModel, get_cosine_schedule_with_warmup

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Configuration

In [6]:
# === Paths (EDITED) ===
DATASET_ROOT = Path(r"D:\Swinburne\Degree3_S2\COS30082_AML\GroupAssignment\dataset")
TRAIN_LIST       = DATASET_ROOT / "list" / "train.txt"
TEST_LIST        = DATASET_ROOT / "list" / "test.txt"
GROUNDTRUTH_LIST = DATASET_ROOT / "list" / "groundtruth.txt"

# === Hyperparameters ===
MODEL_NAME = "facebook/vit-mae-large"
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
LR = 5e-5
WEIGHT_DECAY = 0.05
WARMUP_RATIO = 0.1
NUM_WORKERS = 4
SEED = 42
AMP = True  # mixed precision

# === Check paths ===
print("=== Path Check ===")
for p in [DATASET_ROOT, TRAIN_LIST, TEST_LIST, GROUNDTRUTH_LIST]:
    print(f"{p} -> {'OK' if p.exists() else 'MISSING'}")

# === Stage 1: Check model loading ===
print("\n[Stage 1] Checking model download/loading ...")
t0 = time.time()
try:
    from transformers import AutoImageProcessor, ViTMAEModel
    processor = AutoImageProcessor.from_pretrained(MODEL_NAME, use_fast=True)
    backbone = ViTMAEModel.from_pretrained(MODEL_NAME)
    print(f"✅ Model loaded successfully in {time.time()-t0:.1f}s")
    print(f"Hidden size: {backbone.config.hidden_size}")
except Exception as e:
    print(f"❌ Model load failed after {time.time()-t0:.1f}s")
    print("Error:", e)

# === Stage 2: Check data file reading ===
print("\n[Stage 2] Checking dataset reading ...")
try:
    with open(TRAIN_LIST, "r", encoding="utf-8") as f:
        train_lines = sum(1 for _ in f)
    with open(TEST_LIST, "r", encoding="utf-8") as f:
        test_lines = sum(1 for _ in f)
    with open(GROUNDTRUTH_LIST, "r", encoding="utf-8") as f:
        gt_lines = sum(1 for _ in f)
    print(f"✅ Data files read successfully.")
    print(f"Train lines: {train_lines}, Test lines: {test_lines}, Groundtruth lines: {gt_lines}")
except Exception as e:
    print("❌ Dataset read failed.")
    print("Error:", e)

# === Device check ===
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n[Stage 3] Using device: {device}")

=== Path Check ===
D:\Swinburne\Degree3_S2\COS30082_AML\GroupAssignment\dataset -> OK
D:\Swinburne\Degree3_S2\COS30082_AML\GroupAssignment\dataset\list\train.txt -> OK
D:\Swinburne\Degree3_S2\COS30082_AML\GroupAssignment\dataset\list\test.txt -> OK
D:\Swinburne\Degree3_S2\COS30082_AML\GroupAssignment\dataset\list\groundtruth.txt -> OK

[Stage 1] Checking model download/loading ...
✅ Model loaded successfully in 2.9s
Hidden size: 1024

[Stage 2] Checking dataset reading ...
✅ Data files read successfully.
Train lines: 4744, Test lines: 207, Groundtruth lines: 207

[Stage 3] Using device: cuda


# Data List Utilities

In [7]:
def read_list_with_labels(list_path: Path) -> List[Tuple[str, int]]:
    """
    Read lines of form: 'relative/path label' -> returns [(rel_path, label_int), ...]
    """
    pairs = []
    with open(list_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) != 2:
                continue
            rel_path, label_str = parts
            try:
                label = int(label_str)
            except:
                continue
            pairs.append((rel_path, label))
    return pairs


def read_list_paths(list_path: Path) -> List[str]:
    """
    Read lines like: 'relative/path' (no labels).
    """
    paths = []
    with open(list_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) < 1:
                continue
            rel_path = parts[0]
            paths.append(rel_path)
    return paths


def build_groundtruth_map(gt_path: Path) -> Dict[str, int]:
    """
    Build mapping {rel_path: label} from groundtruth.txt.
    """
    mapping = {}
    with open(gt_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) != 2:
                continue
            rel_path, label_str = parts
            try:
                mapping[rel_path] = int(label_str)
            except:
                continue
    return mapping

print("Files line counts ->",
      "train:", sum(1 for _ in open(TRAIN_LIST, "r", encoding="utf-8")),
      "| test:", sum(1 for _ in open(TEST_LIST, "r", encoding="utf-8")),
      "| groundtruth:", sum(1 for _ in open(GROUNDTRUTH_LIST, "r", encoding="utf-8")))

Files line counts -> train: 4744 | test: 207 | groundtruth: 207


# Transforms (Augmentation)

In [8]:
import torchvision.transforms as T

# Herbarium: heavy augmentation
train_herbarium_transform = T.Compose([
    T.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
    T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomRotation(45),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Photo: light augmentation
train_photo_transform = T.Compose([
    T.RandomResizedCrop(IMAGE_SIZE),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Validation/Test: center-crop evaluation
eval_transform = T.Compose([
    T.Resize(IMAGE_SIZE + 32),
    T.CenterCrop(IMAGE_SIZE),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def pick_transform(rel_path: str, train: bool = True):
    """
    Heuristically choose transform by path keyword ('herbarium' or 'photo').
    Defaults to 'photo' style if not matched.
    """
    low = rel_path.lower()
    if "herbarium" in low:
        return train_herbarium_transform if train else eval_transform
    if "photo" in low:
        return train_photo_transform if train else eval_transform
    return train_photo_transform if train else eval_transform


# Dataset & DataLoader

In [9]:
class TxtDataset(Dataset):
    def __init__(self, root: Path, pairs: List[Tuple[str, int]], train: bool):
        self.root = root
        self.samples = pairs
        self.train = train

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        rel_path, label = self.samples[idx]
        img_path = self.root / rel_path
        with Image.open(img_path).convert("RGB") as img:
            transform = pick_transform(rel_path, train=self.train)
            img_t = transform(img)
        return img_t, label


# Build train pairs and remap labels to [0..C-1]
train_pairs = read_list_with_labels(TRAIN_LIST)
labels = sorted(set(lbl for _, lbl in train_pairs))
label_to_idx = {lbl: i for i, lbl in enumerate(labels)}
idx_to_label = {i: lbl for lbl, i in label_to_idx.items()}
train_pairs_remap = [(p, label_to_idx[lbl]) for p, lbl in train_pairs]
num_classes = len(labels)
print(f"Num classes: {num_classes}")

train_ds = TxtDataset(DATASET_ROOT, train_pairs_remap, train=True)

# Build validation pairs from test.txt + groundtruth.txt
test_paths = read_list_paths(TEST_LIST)
gt_map = build_groundtruth_map(GROUNDTRUTH_LIST)

val_pairs = []
missing = 0
for rel in test_paths:
    if rel in gt_map and gt_map[rel] in label_to_idx:
        val_pairs.append((rel, label_to_idx[gt_map[rel]]))
    else:
        missing += 1
print(f"Validation pairs: {len(val_pairs)}, missing or unseen labels: {missing}")

val_ds = TxtDataset(DATASET_ROOT, val_pairs, train=False)

# Tip: if you see hanging in notebooks on Windows, try NUM_WORKERS = 0
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

# Quick smoke test: check a few sample paths exist
def check_txt_paths(root: Path, pairs: List[Tuple[str,int]], k: int = 20):
    missing = 0
    for rel, _ in pairs[:k]:
        p = root / rel
        if not p.exists():
            print("MISSING:", p)
            missing += 1
    print(f"Checked {min(k, len(pairs))} samples, missing={missing}")

check_txt_paths(DATASET_ROOT, train_pairs, k=30)


Num classes: 100
Validation pairs: 207, missing or unseen labels: 0
Checked 30 samples, missing=0


# Model: MAE Encoder + Linear Head

In [10]:
# Keep processor for potential future use; fast preprocessor is fine
processor = AutoImageProcessor.from_pretrained(MODEL_NAME, use_fast=True)
backbone = ViTMAEModel.from_pretrained(MODEL_NAME)

class MAEClassifier(nn.Module):
    def __init__(self, backbone: ViTMAEModel, num_classes: int):
        super().__init__()
        self.backbone = backbone
        hidden = backbone.config.hidden_size  # 1024 for ViT-L
        self.classifier = nn.Linear(hidden, num_classes)

    def forward(self, x):
        out = self.backbone(pixel_values=x)
        cls = out.last_hidden_state[:, 0]   # [CLS] token
        logits = self.classifier(cls)
        return logits

model = MAEClassifier(backbone, num_classes).to(device)
print(model.classifier)

Linear(in_features=1024, out_features=100, bias=True)


# Optimizer, Scheduler & Loss

In [11]:
# For a linear-probe phase, set requires_grad=False first, then unfreeze later.
for p in model.backbone.parameters():
    p.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

total_steps = EPOCHS * max(1, len(train_loader))
warmup_steps = int(WARMUP_RATIO * total_steps)
scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

scaler = torch.cuda.amp.GradScaler(enabled=AMP)

# Optional: speed up cudnn on fixed input size
torch.backends.cudnn.benchmark = True

C:\Users\User\AppData\Local\Temp\ipykernel_24640\2077627818.py:16: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=AMP)


# Training & Validation Loops

In [ ]:
def run_one_epoch(loader, train=True):
    stage = "TRAIN" if train else "VAL"
    print(f"--> Starting {stage} phase...")
    model.train(train)
    total_loss, correct, total = 0.0, 0, 0

    pbar = tqdm(loader, desc=stage, leave=False, ncols=100)

    for imgs, labels in pbar:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast(enabled=AMP):
            logits = model(imgs)
            loss = criterion(logits, labels)

        if train:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

        total_loss += loss.item() * imgs.size(0)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += imgs.size(0)

        pbar.set_postfix({
            "loss": f"{loss.item():.4f}",
            "acc": f"{(correct / max(total, 1)):.4f}"
        })

    avg_loss = total_loss / max(total, 1)
    acc = correct / max(total, 1)
    print(f"--> Finished {stage}: loss={avg_loss:.4f}, acc={acc:.4f}")
    return avg_loss, acc


# Optional: smoke test
print("Running smoke test to verify one batch...")
imgs0, labels0 = next(iter(train_loader))
print("Smoke batch:", imgs0.shape, labels0.shape)
with torch.no_grad():
    _ = model(imgs0.to(device))
print("Forward pass OK.")

# Training loop with progress prints
best_acc = 0.0
for ep in range(1, EPOCHS + 1):
    print(f"\n========== Epoch {ep}/{EPOCHS} ==========")
    t0 = time.time()

    print("🚀 Training started...")
    tr_loss, tr_acc = run_one_epoch(train_loader, train=True)

    print("🧪 Validation started...")
    va_loss, va_acc = run_one_epoch(val_loader, train=False)

    dt = time.time() - t0
    print(f"[{ep:02d}/{EPOCHS}] train loss {tr_loss:.4f} acc {tr_acc:.4f} | "
          f"val loss {va_loss:.4f} acc {va_acc:.4f} | {dt:.1f}s")

    if va_acc > best_acc:
        print("💾 Saving new best checkpoint ...")
        best_acc = va_acc
        torch.save({
            "model": model.state_dict(),
            "label_to_idx": label_to_idx,
            "idx_to_label": idx_to_label,
            "config": {"model_name": MODEL_NAME, "image_size": IMAGE_SIZE}
        }, "best_mae_classifier.pt")
        print(f"✅ New best val acc: {best_acc:.4f} saved successfully!")

print("\nTraining complete 🎉")

Running smoke test to verify one batch...
